In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

2023-06-08 10:14:40.865400: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 10:14:44.099315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 10:14:44.113540: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 10:14:53.544442: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

# 1. Data

In [5]:
data_fake = pd.read_csv("../raw_data/fake-news.csv")

In [53]:
data_hate= pd.read_csv("../raw_data/hate-speech.csv")

In [54]:
random_seed = 42

data_sub = data_hate.sample(n=1000, random_state=random_seed)

In [55]:
X = data_sub['text']
y = data_sub['label']

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 1. BERT PREPROC - exploring

The model we chose for BERT Embedding works with a specific preprocessor  
it inlcudes tokenization and padding

In [9]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

2023-06-08 10:15:31.821866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-06-08 10:15:31.822873: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-06-08 10:15:31.823098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-06-08 10:15:31.823767: I tensorflow/core/

In [10]:
#text_test = ['ok I see what you mean darling', 'do not do that in the desert or you will die']
X_train_preproc = bert_preprocess_model(X_train)  # it is a dict
print(X_train_preproc.keys())

# input_word_ids : contains the IDs of each token in the input text
# input_mask: indicates which tokens in the input sequence are actual words and which ones are padding tokens
# => tensor of a certain shape, shape[0] is the number of sentences and shape[1] length of padding
# input_type_ids

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])


# 2. BERT Encoder - exploring

In [11]:
bert_encoder_model = hub.KerasLayer(encoder_url)

In [12]:
output = bert_encoder_model(X_train_preproc)

In [13]:
output.keys()

dict_keys(['pooled_output', 'sequence_output', 'encoder_outputs', 'default'])

In [14]:
print(output['pooled_output'].shape) # => num of senteces/rows, size of embedding vector
print(len(output['encoder_outputs'])) # => 12 because we used a small BERT model with 12 encoders
print((output['encoder_outputs'][0].shape)) # => num of senteces/rows, padding size, size of embedding vector

(150, 768)
12
(150, 128, 768)


In [15]:
X_train_encoded = output['pooled_output']
X_train_encoded

<tf.Tensor: shape=(150, 768), dtype=float32, numpy=
array([[-0.7815599 , -0.41604653, -0.6697939 , ..., -0.48963648,
        -0.5472172 ,  0.80348665],
       [-0.60785097, -0.28559744, -0.6791864 , ..., -0.31198895,
        -0.56930023,  0.4917608 ],
       [-0.8159721 , -0.55098385, -0.7703171 , ..., -0.31313154,
        -0.64943296,  0.5353548 ],
       ...,
       [-0.94676465, -0.44679666, -0.77216285, ..., -0.19994205,
        -0.60622716,  0.87176985],
       [-0.81957006, -0.45411915, -0.6177753 , ..., -0.3751994 ,
        -0.6901966 ,  0.81343234],
       [-0.7721559 , -0.3242671 , -0.5379801 , ..., -0.02808042,
        -0.5813943 ,  0.50213766]], dtype=float32)>

# The Model

In [39]:
# BERT layers

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
output = bert_encoder_model(preprocessed_text)

# Neural Network layers

l = tf.keras.layers.Dropout(0.1, name='dropout')(output['pooled_output'])
l = tf.keras.layers.Dense(64, activation='relu', name='hidden')(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# Construct final model

model = tf.keras.Model(inputs=[text_input], outputs=[l])


2023-06-08 10:52:57.045056: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-06-08 10:52:57.191031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [?,128]
	 [[{{node Placeholder_1}}]]
2023-06-08 10:52:57.191180: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32 and shape [?,128]
	 [[{{node

In [58]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [41]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [57]:
#from tensorflow.keras.callbacks import EarlyStopping

#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#es = EarlyStopping(monitor='val_loss', patience=3)

#history = model.fit(X_train, y_train, epochs=5, #validation_data=(X_val, y_val), #callbacks=[es])

history = model.fit(X_train, y_train, epochs=5)

Epoch 1/5
24/24 [==============================] - 138s 6s/step - loss: 0.6920 - accuracy: 0.5307 - precision: 0.5459 - recall: 0.5375
Epoch 2/5
24/24 [==============================] - 141s 6s/step - loss: 0.6932 - accuracy: 0.5480 - precision: 0.5569 - recall: 0.6072
Epoch 3/5
24/24 [==============================] - 137s 6s/step - loss: 0.6790 - accuracy: 0.5760 - precision: 0.5782 - recall: 0.6589
Epoch 4/5
24/24 [==============================] - 138s 6s/step - loss: 0.6707 - accuracy: 0.5893 - precision: 0.5947 - recall: 0.6408
Epoch 5/5
24/24 [==============================] - 139s 6s/step - loss: 0.6653 - accuracy: 0.5960 - precision: 0.5933 - recall: 0.6899
